In [ ]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch

In [ ]:
data_folder = Path("./raw_data")

In [ ]:
train_df = pd.read_csv("./raw_data/train.csv")
test_df = pd.DataFrame({"filename": [f'{i}.jpg' for i in range(1434)]})

In [ ]:
test_img = ImageList.from_df(test_df, path=data_folder, folder='test')

In [ ]:
trfm = get_transforms(do_flip=True,
                      max_rotate=10.0, 
                      max_zoom=1.25, 
                      max_lighting=0.2, 
                      max_warp=0.2, 
                      p_affine=0.65, 
                      p_lighting=0.55)

train_img = (ImageList.from_df(train_df, path=data_folder, folder='train')
    .split_by_rand_pct(0.01)
    .label_from_df()
    .add_test(test_img)
    .transform(trfm, size=128)
    .databunch(path='.', bs=128, device= torch.device('cuda:0'))
    .normalize(imagenet_stats)
)

In [ ]:
train_img.show_batch(rows=3, figsize=(7,6))

In [ ]:
learn = cnn_learner(train_img, models.densenet121, metrics=[error_rate, accuracy]) 

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
callbacks = [
    callbacks.EarlyStoppingCallback(learn, min_delta=1e-5, patience=3),
    callbacks.SaveModelCallback(learn)
]

learn.callbacks = callbacks

In [ ]:
lr = 1e-02
learn.fit_one_cycle(10, slice(lr))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(7,6))

In [ ]:
learn.load('bestmodel')

In [ ]:
preds, _ = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
test_df['labels'] = np.argmax(preds.numpy(), axis=1)

In [ ]:
sub = test_df.copy()
sub['ID'] = test_df.index
sub[['ID', 'labels']].to_csv('submission.csv', index=False, header=False)

In [ ]:
!head submission.csv